Creating all pair in (shop_id, item_id) for each date_block_num. And if row has not item_count from origin sales data, item_count will fill 0.

In [1]:
# coding: utf-8
import os

import pandas as pd
import numpy as np
import scipy
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.linear_model import LinearRegression
from IPython.display import display

% matplotlib inline

## Config

In [2]:
# trn_path = './data/input/train.csv'
tst_path = './data/input/test.csv'
id_col = 'ID'
label_col = 'item_cnt_month'

submission_path = './data/output/submission/extend_all_train_datasets.csv'
output_id_col = id_col
output_label_col = label_col

## Load Data

In [3]:
item_cate_df = pd.read_csv('./data/input/item_categories.csv')
item_df = pd.read_csv('./data/input/items.csv')
sales_df = pd.read_csv('./data/input/sales_train.csv')
shop_df = pd.read_csv('./data/input/shops.csv')

In [4]:
sales_df['revenue'] = sales_df['item_price'] * sales_df['item_cnt_day']
trn_df = sales_df.groupby(['shop_id', 'item_id', 'date_block_num'])['item_cnt_day'].sum().reset_index()
trn_df = trn_df.rename(columns={'item_cnt_day': 'item_cnt_month'})
trn_df['ID'] = 0
trn_df['ds_type'] = 'trn'

tst_df = pd.read_csv(tst_path)
tst_df['date_block_num'] = 34
tst_df['item_cnt_month'] = 0
tst_df['ds_type'] = 'tst'

full_df = pd.concat([trn_df, tst_df])
full_df.head()

,ID,date_block_num,ds_type,item_cnt_month,item_id,shop_id
0,0,1,trn,31.0,30,0
1,0,1,trn,11.0,31,0
2,0,0,trn,6.0,32,0
3,0,1,trn,10.0,32,0
4,0,0,trn,3.0,33,0


## Get grid

(shop_id, item_id) for each date_block_num

In [5]:
for dbn in range(0, 34):
    # create pair in (shop_id, item_id)
    unique_shop_id = full_df[full_df['date_block_num'] == dbn]['shop_id'].copy().drop_duplicates()
    unique_item_id = full_df[full_df['date_block_num'] == dbn]['item_id'].copy().drop_duplicates()
    m_index = pd.MultiIndex.from_product([unique_shop_id, unique_item_id], names=['shop_id', 'item_id'])
    val_df = pd.DataFrame([], index=m_index).reset_index()
    val_df['ID'] = 0
    val_df['date_block_num'] = dbn
    # distinguish origin train data
    val_df['ds_type'] = 'add_val'
    # remove pair (shop_id, item_id) exists in full_df
    origin_33_df = full_df[full_df['date_block_num'] == dbn][['item_id', 'shop_id', 'item_cnt_month']]
    val_df = val_df.merge(origin_33_df, how='left', on=['item_id', 'shop_id'])
    val_df = val_df[val_df.item_cnt_month.isnull()]
    # no item_count then fill 0
    val_df['item_cnt_month'] = 0

    # merge to full_df
    full_df = pd.concat([full_df, val_df])

## Features

In [6]:
feature_columns = []

def add_features(features):
    if not isinstance(features, list):
        features = [features]
    global feature_columns
    feature_columns.extend([f for f in features if f not in feature_columns])
    
def remove_features(features):
    if not isinstance(features, list):
        features = [features]
    global feature_columns
    feature_columns = [f for f in feature_columns if f not in features]

**shop_id, item_id**

In [7]:
# add_features(['shop_id', 'item_id'])

**datetime info**

In [8]:
full_df['dt_year'] = full_df['date_block_num'] // 12 + 2013
full_df['dt_month'] = full_df['date_block_num'] % 12 + 1
add_features(['dt_year', 'dt_month'])

**item_df**

In [9]:
# item_name's words count
item_df['item_name_words_count'] = item_df['item_name'].map(lambda x: len(x.split(' ')))

# add to full_df
full_df = full_df.merge(item_df, how='left', on='item_id')

# add feautures
add_features(['item_category_id', 'item_name_words_count'])

**mean encoding**

In [10]:
global_mean = full_df[full_df['date_block_num'] < 34]['item_cnt_month'].mean()

In [11]:
def get_aggragation_feature(df, groupby_cols, agg_col, fillna_value):
    gb = df[df['ds_type'] == 'trn'].groupby(groupby_cols)[agg_col]
    fname_fmt = '-'.join(groupby_cols+[agg_col]) + ':%s'
    agg_df = pd.DataFrame({
            fname_fmt%'mean': gb.mean(),
#             fname_fmt%'median': gb.median(),
#             fname_fmt%'max': gb.max(),
#             fname_fmt%'min': gb.min(),
        })
    new_df = df.join(agg_df, on=groupby_cols).fillna(global_mean)
    return new_df, agg_df.columns.tolist()

In [12]:
groupby_cols_list = [
    ['shop_id', 'item_id'],
    ['shop_id'],
    ['item_id'],
    ['date_block_num'],
#     ['dt_year'],
    ['dt_month'],
    ['item_category_id'],
]
for groupby_cols in groupby_cols_list:
    full_df, new_feats = get_aggragation_feature(full_df, groupby_cols, 'item_cnt_month', global_mean)
    add_features(new_feats)

**lag features**

In [13]:
shift_range = [1,2,3,6,12]
shift_features = ['shop_id-item_id-item_cnt_month:mean', 'shop_id-item_cnt_month:mean', 'item_id-item_cnt_month:mean']

for shift_month in shift_range:
    tmp_df = full_df[['shop_id', 'item_id', 'date_block_num'] + shift_features].copy()
    tmp_df['date_block_num'] = tmp_df['date_block_num'] + shift_month
    for f in shift_features:
        new_f = f + '_lag_' + str(shift_month)
        tmp_df = tmp_df.rename(columns={f: new_f})
        add_features(new_f)
    full_df = full_df.merge(tmp_df, how='left', on=['shop_id', 'item_id', 'date_block_num']).fillna(0)

In [14]:
# remove 2013
full_df = full_df[full_df['date_block_num'] > 12]

**show all features**

In [15]:
feature_columns

['dt_year',
 'dt_month',
 'item_category_id',
 'item_name_words_count',
 'shop_id-item_id-item_cnt_month:mean',
 'shop_id-item_cnt_month:mean',
 'item_id-item_cnt_month:mean',
 'date_block_num-item_cnt_month:mean',
 'dt_month-item_cnt_month:mean',
 'item_category_id-item_cnt_month:mean',
 'shop_id-item_id-item_cnt_month:mean_lag_1',
 'shop_id-item_cnt_month:mean_lag_1',
 'item_id-item_cnt_month:mean_lag_1',
 'shop_id-item_id-item_cnt_month:mean_lag_2',
 'shop_id-item_cnt_month:mean_lag_2',
 'item_id-item_cnt_month:mean_lag_2',
 'shop_id-item_id-item_cnt_month:mean_lag_3',
 'shop_id-item_cnt_month:mean_lag_3',
 'item_id-item_cnt_month:mean_lag_3',
 'shop_id-item_id-item_cnt_month:mean_lag_6',
 'shop_id-item_cnt_month:mean_lag_6',
 'item_id-item_cnt_month:mean_lag_6',
 'shop_id-item_id-item_cnt_month:mean_lag_12',
 'shop_id-item_cnt_month:mean_lag_12',
 'item_id-item_cnt_month:mean_lag_12']

## Level1 Ensemble

**Valdation**

In [16]:
dates = full_df['date_block_num'].copy()

dates_trn = dates[dates <= 33]
dates_tst = dates[dates == 34]

trn_df = full_df[dates <= 33]
tst_df = full_df[dates == 34]

X_trn = trn_df[feature_columns]
y_trn = trn_df[label_col]
X_tst = tst_df[feature_columns]
y_tst = tst_df[label_col]

# # scale
# scaler = preprocessing.StandardScaler()
# X_trn = scaler.fit_transform(X_trn)
# X_val = scaler.transform(X_val)

**Score**

In [17]:
# score
def rmse(y, y_pred):
    return np.sqrt(metrics.mean_squared_error(y, y_pred))

**ElasticNet parameters tuning**

In [18]:
# param_grid = {
# #     'alpha': [0.001, 0.01, 0.1, 1, 10],
#     'alpha': [1],
# #     'l1_ratio': [0, 0.25, 0.5, 0.75, 1],
#     'l1_ratio': [0.5],
# }

# best_score = 9999
# best_param = None
# for param in model_selection.ParameterGrid(param_grid):
#     print(param)
#     clf = ElasticNet(**param)
#     clf.fit(X_trn, y_trn)
#     s = rmse(y_val, clf.predict(X_val).clip(0,20))
#     if s < best_score:
#         best_score = s
#         best_param = param
#         val_df['pred'] = clf.predict(X_val).clip(0,20)
#     print(s)

# level1_en = ElasticNet(**best_param)

en_params = {
    'alpha': 1,
    'l1_ratio': 0.5,
}

**LightGBM parameters tuning**

In [19]:
lgb_params = {
    'feature_fraction': 0.75,
    'metric': 'rmse',
    'nthread': 1, 
    'min_data_in_leaf': 2**7, 
    'bagging_fraction': 0.75, 
    'learning_rate': 0.03, 
    'objective': 'mse', 
    'bagging_seed': 2**7, 
    'num_leaves': 2**7,
    'bagging_freq': 1,
    'verbose': 0,
}

**Get level2 train data**

In [20]:
level2_block_num = [28, 29, 30, 31, 32, 33]
dates_trn_level2 = dates_trn[dates_trn.isin(level2_block_num)]
y_trn_level2 = y_trn[dates_trn.isin(level2_block_num)]

In [21]:
X_trn_level2 = np.zeros([y_trn_level2.shape[0], 2])

for cur_block_num in level2_block_num:
    print(cur_block_num)

    X_trn_i = X_trn[dates_trn < cur_block_num]
    y_trn_i = y_trn[dates_trn < cur_block_num]
    X_tst_i = X_trn[dates_trn == cur_block_num]
    
    en = ElasticNet(**en_params)
    en.fit(X_trn_i, y_trn_i)
    X_trn_level2[dates_trn_level2 == cur_block_num, 0] = en.predict(X_tst_i)
    
    lgb_model = lgb.train(lgb_params, lgb.Dataset(X_trn_i, label=y_trn_i), 100)
    X_trn_level2[dates_trn_level2 == cur_block_num, 1] = lgb_model.predict(X_tst_i)

28
29
30
31
32
33


**Get level2 test data**

In [22]:
en = ElasticNet(**en_params)
en.fit(X_trn, y_trn)
X_tst_level2_en = en.predict(X_tst)

lgb = lgb.train(lgb_params, lgb.Dataset(X_trn, label=y_trn), 100)
X_tst_level2_lgb = lgb.predict(X_tst)

X_tst_level2 = np.c_[X_tst_level2_en, X_tst_level2_lgb]

## Level2 Ensemble: Stacking

In [25]:
lr = LinearRegression()
lr.fit(X_trn_level2, y_trn_level2)
predictions = lr.predict(X_tst_level2)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [30]:
rmse(y_trn_level2, lr.predict(X_trn_level2))

3.3070180165433514

## Predict

In [31]:
res_df = pd.DataFrame(predictions.clip(0,20), columns=[output_label_col])
res_df[output_id_col] = tst_df[output_id_col].astype(int).values
res_df[[output_id_col, output_label_col]].to_csv(submission_path, index=False)